Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import range
from __future__ import print_function
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [5]:
# Subset the training data for faster turnaround.
train_subset = 50000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_train_dataset = tf.constant(train_dataset[:train_subset,:])
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    biases = tf.Variable(tf.zeros([num_labels]))
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset,weights)+biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)

Let's run this computation and iterate:

In [6]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.290522
Training accuracy: 11.4%
Validation accuracy: 12.8%
Loss at step 100: 2.356305
Training accuracy: 71.3%
Validation accuracy: 71.2%
Loss at step 200: 1.974864
Training accuracy: 73.8%
Validation accuracy: 74.0%
Loss at step 300: 1.762495
Training accuracy: 74.9%
Validation accuracy: 74.9%
Loss at step 400: 1.615204
Training accuracy: 75.5%
Validation accuracy: 75.3%
Loss at step 500: 1.504139
Training accuracy: 76.0%
Validation accuracy: 75.7%
Loss at step 600: 1.416431
Training accuracy: 76.4%
Validation accuracy: 76.0%
Loss at step 700: 1.344858
Training accuracy: 76.7%
Validation accuracy: 76.3%
Loss at step 800: 1.284997
Training accuracy: 77.0%
Validation accuracy: 76.6%
Test accuracy: 83.8%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [7]:
batch_size = 512

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    
    # Variables.
    biases = tf.Variable(tf.zeros([num_labels]))
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights)+biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+biases)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset,weights)+biases)

Let's run it:

In [8]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  
    tf.initialize_all_variables().run()
    print("Initialized")
  
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step*batch_size)%(train_labels.shape[0]-batch_size)
    
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
    
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_labels}
    
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.354591
Minibatch accuracy: 8.8%
Validation accuracy: 9.9%
Test accuracy: 9.6%
Minibatch loss at step 500: 1.388033
Minibatch accuracy: 76.8%
Validation accuracy: 75.6%
Test accuracy: 82.5%
Minibatch loss at step 1000: 1.089339
Minibatch accuracy: 78.1%
Validation accuracy: 76.8%
Test accuracy: 83.8%
Minibatch loss at step 1500: 1.318462
Minibatch accuracy: 75.0%
Validation accuracy: 77.2%
Test accuracy: 84.7%
Minibatch loss at step 2000: 1.084758
Minibatch accuracy: 77.9%
Validation accuracy: 77.9%
Test accuracy: 85.2%
Minibatch loss at step 2500: 0.766470
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Test accuracy: 85.5%
Minibatch loss at step 3000: 0.835090
Minibatch accuracy: 77.7%
Validation accuracy: 79.1%
Test accuracy: 86.3%
Minibatch loss at step 3500: 0.864222
Minibatch accuracy: 78.7%
Validation accuracy: 79.4%
Test accuracy: 86.6%
Minibatch loss at step 4000: 0.777528
Minibatch accuracy: 80.9%
Validation accuracy: 79.5%
Test ac

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [11]:
batch_size = 512
num_hidden = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)    
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_hidden_units = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    
    # Variables.
    biases1 = tf.Variable(tf.zeros([num_hidden]))
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size,num_hidden]))

    biases2 = tf.Variable(tf.zeros([num_labels]))
    weights2 = tf.Variable(tf.truncated_normal([num_hidden,num_labels]))
    
    # Training computation.
    tf_hidden_units = tf.nn.relu(tf.matmul(tf_train_dataset, weights1)+biases1)
    
    logits = tf.matmul(tf_hidden_units, weights2)+biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights1)+biases1),
                                              weights2)+biases2)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights1)+biases1),
                                               weights2)+biases2)

In [12]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  
    tf.initialize_all_variables().run()
    print("Initialized")
  
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step*batch_size)%(train_labels.shape[0]-batch_size)
    
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
    
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_labels}
    
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 413.537567
Minibatch accuracy: 5.3%
Validation accuracy: 45.1%
Test accuracy: 49.1%
Minibatch loss at step 500: 7.049460
Minibatch accuracy: 85.2%
Validation accuracy: 81.7%
Test accuracy: 88.0%
Minibatch loss at step 1000: 3.375820
Minibatch accuracy: 86.7%
Validation accuracy: 84.0%
Test accuracy: 90.7%
Minibatch loss at step 1500: 10.526062
Minibatch accuracy: 82.4%
Validation accuracy: 81.1%
Test accuracy: 87.5%
Minibatch loss at step 2000: 3.784374
Minibatch accuracy: 85.2%
Validation accuracy: 81.7%
Test accuracy: 88.2%
Minibatch loss at step 2500: 3.055962
Minibatch accuracy: 86.3%
Validation accuracy: 83.6%
Test accuracy: 90.3%
Minibatch loss at step 3000: 1.348005
Minibatch accuracy: 88.3%
Validation accuracy: 85.0%
Test accuracy: 91.5%
Minibatch loss at step 3500: 2.082696
Minibatch accuracy: 87.3%
Validation accuracy: 82.4%
Test accuracy: 88.5%
Minibatch loss at step 4000: 2.480327
Minibatch accuracy: 86.5%
Validation accuracy: 84.7%
Tes

In [13]:
# load the pickle file to continue analysis
data = pickle.load(open('notMNISTClean.pickle','rb'))

testLabels1 = data['testLabels']
validLabels1 = data['validLabels']

testDataset1 = data['testDataset']
validDataset1 = data['validDataset']

In [18]:
# normalize and prepare the datasets
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

testDataset1,testLabels1 = reformat(testDataset1,testLabels1)
validDataset1,validLabels1 = reformat(validDataset1,validLabels1)

print('Test set', testDataset1.shape, testLabels1.shape)
print('Validation set', validDataset1.shape, validLabels1.shape)

Test set (8689, 784) (8689, 10)
Validation set (8849, 784) (8849, 10)


In [19]:
batch_size = 512
num_hidden = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tfTestClean = tf.constant(testDataset1)
    tfValidClean = tf.constant(validDataset1)    
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)    
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_hidden_units = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    
    # Variables.
    biases1 = tf.Variable(tf.zeros([num_hidden]))
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size,num_hidden]))

    biases2 = tf.Variable(tf.zeros([num_labels]))
    weights2 = tf.Variable(tf.truncated_normal([num_hidden,num_labels]))
    
    # Training computation.
    tf_hidden_units = tf.nn.relu(tf.matmul(tf_train_dataset, weights1)+biases1)
    
    logits = tf.matmul(tf_hidden_units, weights2)+biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    testPredClean = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tfTestClean,weights1)+biases1),
                                            weights2)+biases2)
    validPredClean = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tfValidClean,weights1)+biases1),
                                             weights2)+biases2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weights1)+biases1),
                                              weights2)+biases2)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weights1)+biases1),
                                               weights2)+biases2)

In [20]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  
    tf.initialize_all_variables().run()
    print("Initialized")
  
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step*batch_size)%(train_labels.shape[0]-batch_size)
    
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
    
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_labels}
    
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%% \n" % accuracy(predictions, batch_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
            print("Test accuracy clean: %.1f%% \n" % accuracy(testPredClean.eval(), testLabels1))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Validation accuracy clean: %.1f%% \n" % accuracy(validPredClean.eval(), validLabels1))

Initialized
Minibatch loss at step 0: 283.795776
Minibatch accuracy: 13.3% 

Test accuracy: 39.4%
Test accuracy clean: 37.8% 

Validation accuracy: 36.1%
Validation accuracy clean: 34.5% 

Minibatch loss at step 500: 6.252064
Minibatch accuracy: 85.9% 

Test accuracy: 89.8%
Test accuracy clean: 88.7% 

Validation accuracy: 83.1%
Validation accuracy clean: 81.7% 

Minibatch loss at step 1000: 3.547809
Minibatch accuracy: 84.4% 

Test accuracy: 91.0%
Test accuracy clean: 90.0% 

Validation accuracy: 84.4%
Validation accuracy clean: 83.0% 

Minibatch loss at step 1500: 10.362753
Minibatch accuracy: 80.9% 

Test accuracy: 89.1%
Test accuracy clean: 88.0% 

Validation accuracy: 81.9%
Validation accuracy clean: 80.5% 

Minibatch loss at step 2000: 3.707421
Minibatch accuracy: 85.2% 

Test accuracy: 89.3%
Test accuracy clean: 88.3% 

Validation accuracy: 82.5%
Validation accuracy clean: 81.0% 

Minibatch loss at step 2500: 2.651877
Minibatch accuracy: 85.9% 

Test accuracy: 90.9%
Test accurac